# Coordinate Agent Collaboration using Agent Group Chat

In this sample, we will explore how to use AgentGroupChat to coordinate collboration of two different agents working to review and rewrite user provided content. Each agent is assigned a distinct role:

* Reviewer: Reviews and provides direction to Writer.
* Writer: Updates user content based on Reviewer input.

The approach will be broken down step-by-step to high-light the key parts of the coding process.


## Creating the required Azure resources

You can run the powershell script `create-azure-ai-resources.ps1` to create the following resources:
* Azure AI Services with Hub and Project
* GPT-4o model

## Installing the required packages

Import Semantic Kernel SDK from pypi.org

In [ ]:
%pip install -U semantic-kernel --quiet

Check the current version of Semantic Kernel.

In [ ]:
from semantic_kernel import __version__

__version__

Load the environment variables to connect to the LLM model.

In [ ]:
from dotenv import load_dotenv
import os

if os.path.exists(".env"):
    load_dotenv(override=True)

## Creating a chat completion service

>Note: The AzureChatCompletion service also supports Microsoft Entra authentication. If you don't provide an API key, the service will attempt to authenticate using the Entra token.

In [ ]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)

Let's try chat with LLM model.

In [ ]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.contents.chat_history import ChatHistory

execution_settings = AzureChatPromptExecutionSettings()

system_message = "You are a helpful assistant that can answer questions and provide information."

chat_history = ChatHistory(system_message=system_message)

chat_history.add_user_message("Hello, how are you?")

response = await chat_completion.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

print(response)

## Setup the kernel

Prior to creating any ChatCompletionAgent, the configuration settings, plugins, and Kernel must be initialized.

Initialize the kernel object:

In [ ]:
from semantic_kernel import Kernel

kernel = Kernel()

kernel.add_service(service=chat_completion)


## Agent Definition

We will declare the agent names as `Reviewer` and `Writer`.

In [ ]:
REVIEWER_NAME = "Reviewer"
WRITER_NAME = "Writer"

## Defining the reviewer agent

Here the Reviewer is given the role of responding to user input, providing direction to the Writer agent, and verifying result of the Writer agent.

In [ ]:
from semantic_kernel.agents import ChatCompletionAgent

agent_reviewer = ChatCompletionAgent(
        kernel=kernel,
        name=REVIEWER_NAME,
        instructions="""
Your responsibility is to review and identify how to improve user provided content.
If the user has provided input or direction for content already provided, specify how to address this input.
Never directly perform the correction or provide an example.
Once the content has been updated in a subsequent response, review it again until it is satisfactory.

RULES:
- Only identify suggestions that are specific and actionable.
- Verify previous suggestions have been addressed.
- Never repeat previous suggestions.
""",
)

## Defining the writer agent

The Writer agent is similiar. It is given a single-purpose task, follow direction and rewrite the content.

In [ ]:
agent_writer = ChatCompletionAgent(
        kernel=kernel,
        name=WRITER_NAME,
        instructions="""
Your sole responsibility is to rewrite content according to review suggestions.
- Always apply all review directions.
- Always revise the content in its entirety without explanation.
- Never address the user.
""",
    )

## Chat Definition

Defining the `AgentGroupChat` requires considering the strategies for selecting the Agent turn and determining when to exit the Chat loop. For both of these considerations, we will define a Kernel Prompt Function.

The first to reason over Agent selection:

In [ ]:
from semantic_kernel.functions import KernelFunctionFromPrompt

selection_function = KernelFunctionFromPrompt(
    function_name="selection", 
    prompt=f"""
Examine the provided RESPONSE and choose the next participant.
State only the name of the chosen participant without explanation.
Never choose the participant named in the RESPONSE.

Choose only from these participants:
- {REVIEWER_NAME}
- {WRITER_NAME}

Rules:
- If RESPONSE is user input, it is {REVIEWER_NAME}'s turn.
- If RESPONSE is by {REVIEWER_NAME}, it is {WRITER_NAME}'s turn.
- If RESPONSE is by {WRITER_NAME}, it is {REVIEWER_NAME}'s turn.

RESPONSE:
{{{{$lastmessage}}}}
"""
)

The second will evaluate when to exit the Chat loop:

In [ ]:
termination_keyword = "yes"

termination_function = KernelFunctionFromPrompt(
    function_name="termination", 
    prompt=f"""
Examine the RESPONSE and determine whether the content has been deemed satisfactory.
If the content is satisfactory, respond with a single word without explanation: {termination_keyword}.
If specific suggestions are being provided, it is not satisfactory.
If no correction is suggested, it is satisfactory.

RESPONSE:
{{{{$lastmessage}}}}
"""
)

Both of these Strategies will only require knowledge of the most recent Chat message. This will reduce token usage and help improve performance:

In [ ]:
from semantic_kernel.contents import ChatHistoryTruncationReducer

history_reducer = ChatHistoryTruncationReducer(target_count=1)

Finally we are ready to bring everything together in our `AgentGroupChat` definition.

Creating `AgentGroupChat` involves:

1. Include both agents in the constructor.
2. Define a `KernelFunctionSelectionStrategy` using the previously defined `KernelFunction` and Kernel instance.
3. Define a `KernelFunctionTerminationStrategy` using the previously defined `KernelFunction` and Kernel instance.

Notice that each strategy is responsible for parsing the `KernelFunction` result.

In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)

groupChat = AgentGroupChat(
    agents=[agent_reviewer, agent_writer],
    selection_strategy=KernelFunctionSelectionStrategy(
        initial_agent=agent_reviewer,
        function=selection_function,
        kernel=kernel,
        result_parser=lambda result: str(result.value[0]).strip() if result.value[0] is not None else WRITER_NAME,
        history_variable_name="lastmessage",
        history_reducer=history_reducer,
    ),
    termination_strategy=KernelFunctionTerminationStrategy(
        agents=[agent_reviewer],
        function=termination_function,
        kernel=kernel,
        result_parser=lambda result: termination_keyword in str(result.value[0]).lower(),
        history_variable_name="lastmessage",
        maximum_iterations=10,
        history_reducer=history_reducer,
    ),
)

The lastmessage `history_variable_name` corresponds with the `KernelFunctionSelectionStrategy` and the `KernelFunctionTerminationStrategy` prompt that was defined above. This is where the last message is placed when rendering the prompt.

## The Chat Loop

At last, we are able to coordinate the interaction between the user and the AgentGroupChat. Start by creating creating an empty loop.

>Note: Unlike the other examples, no external history or thread is managed. AgentGroupChat manages the conversation history internally.

In [ ]:
groupChat.is_complete = False
await groupChat.reset()

user_input = "Rozes are red, violetz are blue."
user_input = """
Semantic Kernel (SK) is an open-source SDK that enables developers to build and orchestrate 
complex AI workflows that involve natural language processing (NLP) and machine learning models.
It provies a flexible platform for integrating AI capabilities such as semantic search, 
text summarization, and dialogue systems into applications. 
With SK, you can easily combine different AI services and models, define their relationships, 
and orchestrate interactions between them.
"""

# Add the current user_input to the chat
await groupChat.add_chat_message(message=user_input)

try:
    async for response in groupChat.invoke():
        if response is None or not response.name:
            continue
        print()
        print(f"# {response.name.upper()}:\n{response.content}")
except Exception as e:
    print(f"Error during chat invocation: {e}")

groupChat.is_complete = True

You can also interact with the chat by providing text input. 

In [ ]:
is_complete: bool = False
groupChat.is_complete = False
await groupChat.reset()

while not is_complete:
    print()
    user_input = input("User > ").strip()
    if not user_input:
        continue

    if user_input.lower() == "exit":
        is_complete = True
        break

    if user_input.lower() == "reset":
        await groupChat.reset()
        print("[Conversation has been reset]")
        continue

    # Try to grab files from the script's current directory
    if user_input.startswith("@") and len(user_input) > 1:
        file_name = user_input[1:]
        script_dir = os.path.dirname(os.path.abspath(__file__))
        file_path = os.path.join(script_dir, file_name)
        try:
            if not os.path.exists(file_path):
                print(f"Unable to access file: {file_path}")
                continue
            with open(file_path, "r", encoding="utf-8") as file:
                user_input = file.read()
        except Exception:
            print(f"Unable to access file: {file_path}")
            continue

    # Add the current user_input to the chat
    await groupChat.add_chat_message(message=user_input)

    try:
        async for response in groupChat.invoke():
            if response is None or not response.name:
                continue
            print()
            print(f"# {response.name.upper()}:\n{response.content}")
    except Exception as e:
        print(f"Error during chat invocation: {e}")

    # Reset the chat's complete flag for the new conversation round.
    groupChat.is_complete = False

Now let's capture user input within the previous loop. In this case:

* Empty input will be ignored.
* The term exit will signal that the conversation is complete.
* The term reset will clear the AgentGroupChat history.
* Any term starting with @ will be treated as a file-path whose content will be provided as input.

Valid input will be added to the `AgentGroupChat` as a User message.

To initate the Agent collaboration in response to user input and display the Agent responses, invoke the AgentGroupChat; however, first be sure to reset the Completion state from any prior invocation.

>Note: Service failures are being caught and displayed to avoid crashing the conversation loop.

## More resources

https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/examples/example-agent-collaboration?pivots=programming-language-python